<a href="https://colab.research.google.com/github/kuroneko9696/ssg_conv/blob/master/sasage_nohin_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## 納品データコンバーター
 
 

*   エクステンシスで出力されたxxxxx.json形式のデータをお客様の要望に合わせた納品形式にして出力します。
*   素材・洗濯表記・原稿などの商品データと、サイズ毎の採寸情報をまとめてエクセル形式で出力します。
*画像データは非対象


 ### 使い方
* 下の「各種設定」の項目を入力します。入力ルールは別途確認してください。
* 
image.pngをクリックしてコンバーターを開始してください。
* コンバートが終わるとCSVファイルがダウンロードされます。




In [9]:
#@title 納品コンバーター
#納品ファイルを開く(JOSN)
from google.colab import drive
import os

#テスト
#納品ファイルをPCから受け渡すためのダイアログを開く

#@markdown >json形式のファイルをアップロードしてください。
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_name_body = file_name.split(".")[0]  #他のファイル形式で保存する際に使う

print(file_name)
directory = %pwd
myfile = directory+"/"+file_name

import json
jsonfile = open(myfile,"r")

data = json.load(jsonfile)
print("JSONファイルを読み込みました。--次へ進む--")
  


#---------------------------------------------------------
#@title ②読み込んだJSONの解析
#print(data)
saisun_data = []
genko_data =[]
nohin_data =[]
data_body = data["request"]
print(len(data_body),"件のレコード")

for d in data_body:
    saisun_data2 = []
    #基本情報
    id = d["ID"] 
    customer_code = d["customer_code"] 
    brand_code = d["brand_code"]
    pm_name = d["pm_name"] 
    studio_name = d["studio_name"] 

    #商品情報
    temporary_type = d["temporary_type"] 
    maker_type = d["maker_type"] 
    product_name = d["product_name"] 
    brand_name = d["brand_name"] 
    sex = d["sex"] 
    product_memo = d["product_memo"] 
    product_management_memo = d["product_management_memo"]
    measurement_flag = d["measurement_flag"] 

    #スケジュール
    desired_delivery_date = d["desired_delivery_date"] 
    arrival_scheduled_date = d["arrival_scheduled_date"] 
    delivery_date = d["delivery_date"] 
    box_number = d["box_number"] 
    shipping_date = d["shipping_date"] 
    shipping_staff = d["shipping_staff"] 
    return_date = d["return_date"] 
    
    #---------------ここから顧客ごとにカスタマイズ---------------
    #納品情報　（●は複数になる可能性あり）
    #国（パーツによって異なる場合がある）●
    country = d["country"]
    nohin_texts = []
    for c_parts in country:
        #パーツ名が空の時は、値のみにする
        if c_parts["parts_name"] == "":
            txt = c_parts["value"]
            nohin_texts.append(txt)
        else:
            txt = c_parts["parts_name"]+":"+c_parts["value"]
            nohin_texts.append(txt)
    n_contry_txt = "\n".join(nohin_texts) #リスト化した情報を改行でつなげる

    #クリーニング（パーツによって異なる場合がある）●
    #print("----クリーニング表記---")
    cleaning_types = d["cleaning_types"]
    nohin_texts = []
    for cl_parts in cleaning_types:
        if cl_parts["cleaning_part"] == "":
            txt = cl_parts["value"]
        else:
            txt = cl_parts["cleaning_part"]+":"+cl_parts["value"]
            nohin_texts.append(txt)
    n_cleaning_txt = "\n".join(nohin_texts) #リスト化した情報を改行でつなげる
    #print(n_cleaning_txt)

    #重さ（パーツによって重さや素材が異なる場合がある）●　★未テスト★
    quantities = d["quantities"]
    nohin_texts = []
    for q_parts in quantities:
        if q_parts["quantity_name"] == "":
            txt = q_parts["value"]
        else:
            txt = q_parts["quantity_name"],q_parts["value"]
            nohin_texts.append(txt)
    n_quantities_txt = "、".join(nohin_texts) #リスト化した情報を指定した形式でつなげる

    #quantity_name　（★未テスト★）

    #素材（パーツによって素材、パーセンテージ異なる場合がある）●
    nohin_texts = []
    materials = d["materials"]
    for m_parts in materials:#パーツ事の処理
      m_parts_name  = m_parts["parts_name"] #パーツ名
      #print(m_parts_name)
      parts_texts = []
      for materials in m_parts["item_materials"]:#パーツ内の素材毎の処理
        material = materials["key"] #素材名
        value = materials["value"] #値
        if value =="":#値なし
            txt = material
        else:#値あり
            txt = material+value+"%"
        parts_texts.append(txt)#パーツ内素材のリスト

      parts_text =  "、".join(parts_texts)#リストを指定した文字でつなげる
      parts_text = m_parts_name+" : "+parts_text#つなげた文字とパーツをつなげる
      nohin_texts.append(parts_text)#リストに入れる

      n_materials_txt = "\n".join(nohin_texts) #リスト化した情報を改行でつなげる

    #キーワード（使う？）● ★未テスト★
    #----------------------------------------　
    keywords = d["keywords"]

    #原稿
    product_text = d["product_text"]
    #print(product_text)
    #モデル系
    model_size = d["model_size"]
    model_name = d["model_name"]

    #カラー●
    color = d["color"]
    for c in color:
        #print(c["picture"])
        #print("----画像URLー---")
        for p_url in c["picture"]:
            pass
            #print(p_url["picture_url"])
            #print("----サイズー---")
            #print(c["size"])
        #カラー内のサイズ（なんでカラー内にサイズを入れるかはエクステンシスさんに聞く）
        for s in c["size"]:
            #print("----サイズコードー---")
            #print(s["size_code"])
            #print("----サイズ名ー---")
            #print(s["size_name"])
            #print("----JANー---")
            #print(s["JAN"])
            jan = s["JAN"]
            pass
    #採寸●
    measurement = d["measurement"]
    for sizes in measurement:
      sizename = sizes["size_name"] #size名の取得
      partslist = []
      for size in sizes["item_measurement"]:
        #print(size["item_name"])
        saisunlist = []
        for s_parts in size["size_item_measurement"]:
          #print(s_parts["measurement_name"])
          #print(s_parts["value"])
          txt = s_parts["measurement_name"]+ s_parts["value"]
          #print(txt)
          saisunlist.append(txt)
        n_saisun_txt = "/".join(saisunlist) #リスト化した情報を/でつなげる
        n_saisun_txt = size["item_name"]+":"+n_saisun_txt
        partslist.append(n_saisun_txt)
      saisun_all_text =  "\n".join(partslist) #リスト化した情報を\nでつなげる

      #print(saisun_all_text)
      ##************出力順・内容①採寸***************##
      n_saisun_list =  ["-",maker_type,"-","-",sizename,"-","-",saisun_all_text,"-","-","-"]
      saisun_data2.append(n_saisun_list)
      #JAN	品番	商品名	性別	サイズ名	素材	原産国	部位	採寸データ	原稿	クリーニング	重量
      ##*****************************************##

   
    ##************出力順・内容　②　型***************##
    nohin_all =  ["-",maker_type,product_name,sex,"-",n_materials_txt,n_contry_txt,"-",product_text,n_cleaning_txt,n_quantities_txt]
    #JAN	品番	商品名	性別	サイズ名	素材	原産国	部位	採寸データ	原稿	クリーニング	重量
    print(nohin_all)
    ##***************************##

    nohin_data.append(nohin_all)
    for saisun_data_oneline in saisun_data2:
      #print("**",len(saisun_data_oneline),"**")
      print(saisun_data_oneline)
      nohin_data.append(saisun_data_oneline)
    print("------")


print("JSONの解析が終了しました。--次へ進む--")

#---------------------------------------------------------
#@title ③解析した内容をGoogleDriveにエクセル形式で保存
import csv
import pandas as pd

tmpfile = "tmpfile.csv"

#全部一緒タイプ
with open(tmpfile,"w",encoding="utf-8") as nd:
  writer = csv.writer(nd)
  #writer.writerow(["型番","JAN","商品名","性別","原産国","クリーニング","重量","素材","原稿","部位","サイズ名","採寸データ"])
  writer.writerow(["JAN","品番","商品名","性別","サイズ名","素材","原産国","採寸データ","原稿","クリーニング","重量"])
  #JAN	品番	商品名	性別	サイズ名	素材	原産国	部位	採寸データ	原稿	クリーニング	重量

  for zzz in nohin_data:
    writer.writerow(zzz)
    print(zzz,type(zzz))
#csvをデータフレームに
df = pd.read_csv(tmpfile)
df
#データフレームをエクセルに
df.to_excel(file_name_body+".xlsx",index=False)

#---------------------------------------------------------
#@title ④Excelファイルを見やすいように整形する（※オプション）
#エクセルの装飾（たぶんバグ多い）

import openpyxl
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Alignment

# ブックを取得
book = openpyxl.load_workbook(file_name_body+".xlsx")
# シートを取得
sheet = book['Sheet1']

#罫線
side = Side(style='thin', color='000000')
border = Border(top=side) #上へ罫線

#指定された行を塗りつぶす関数
def fill_row(row):
  row = str(row)
  range_text = "A"+row+":"+"K"+row
  fill = openpyxl.styles.PatternFill(patternType='solid',fgColor='dff7e7')
  renge = sheet[range_text][0]
  for cell in renge:
    cell.fill = fill
    cell.border = border
    #cell.alignment = Alignment(horizontal='right',vertical='top')

#1列目でグループ判別し、色を塗る
lastcol1 = ""
for col1 in tuple(sheet.columns)[1]:  #2行目だけを抽出
  if lastcol1 != col1.value:  #前の行と値が一緒だった場合
    #print(col1.row)
    fill_row(col1.row)
  else:
    pass
  lastcol1 =col1.value

#Header色の設定
fill = openpyxl.styles.PatternFill(patternType='solid',fgColor='6b8e23')
header = sheet["A1:K1"][0]
for hcell in header:
  hcell.fill = fill
  hcell.font = openpyxl.styles.fonts.Font(color='FFFFFF')

#上寄せする
for row in sheet.iter_rows(min_row = 1):
  for col in row:
    col.alignment = Alignment(horizontal='left',vertical='top')


# 保存する
book.save(file_name_body+".xlsx")
from google.colab import files

files.download(file_name_body+".xlsx")


Saving 【エクステンシス用】【アウトレット】3月15日着荷分依頼リスト(new-material).json to 【エクステンシス用】【アウトレット】3月15日着荷分依頼リスト(new-material) (1).json
【エクステンシス用】【アウトレット】3月15日着荷分依頼リスト(new-material).json
JSONファイルを読み込みました。--次へ進む--
3 件のレコード
['-', '55322171001', 'パンツ', 'レディース', '-', '本体 : 綿92%、合成繊維6%、ポリウレタン2%\nパッチ部分 : 牛革', 'チュニジア', '-', 'すっきりと穿けるスキニーシルエット<br>バックポケットに施されたボートのステッチがポイント<br>裾のカットオフが抜け感を演出します。', '', '']
['-', '55322171001', '-', '-', '2', '-', '-', '本体:ウエスト71cm/ヒップ88cm/股上25cm/股下72.5cm/もも周り54cm/裾周り17cm\nパッチ部分:モチーフ（縦）5cm/モチーフ（横）10cm', '-', '-', '-']
['-', '55322171001', '-', '-', '3', '-', '-', '本体:ウエスト77cm/ヒップ93cm/股上26cm/股下73cm/もも周り56cm/裾周り17.5cm\nパッチ部分:モチーフ（縦）6cm/モチーフ（横）11cm', '-', '-', '-']
------
['-', '55322512001', 'ブルゾン', 'レディース', '-', '表地 : ナイロン100%\n一部分 : 合成皮革（ポリウレタン）\n裏地 : ポリエステル100%', 'ベトナム', '-', 'あざやかなターコイズブルーのマウンテンパーカ<br>裏地にメッシュを使用しているので汗ばむ季節にぴったり<br>内側のストラップで背負うことも可能です', '', '']
['-', '55322512001', '-', '-', '2', '-', '-', '本体:着丈61cm/肩幅42cm/バスト109cm/袖丈63cm\n付属品:縦20cm/横50cm', '-', '-', '-']
------
['

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>